In [ ]:
import math
import random
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import time
from prettytable import PrettyTable #pip install prettytable

## Simulated Annealing Algorithm with different methods to test parameters
- Cooling Schedules
  - For additive schedules dont give an 'alpha'
  - For multiplicative schedules give schedule and an alpha
  
- Batch Anneal runs the annealing schedule for mutilple number of 'times' each time with a random starting node.

- Reheat runs annealing multiple times starting from the best solution of the previous annealing. It terminates if better
  solutions are not found.
  
- Sampling methods- Sampling with replacement (this is default value) and without replacement. This is only applicable for    
  multiple annealings,i.e, the Batch Anneal method. Reheat and Anneal dont require a sampling method.  

In [ ]:
class SimAnneal(object):
    def __init__(self, coords, T=-1, alpha=None, stopping_T=-1, cooling_schedule='linear',stopping_iter=-1):
        
        assert cooling_schedule in ['linear','exponential','logarithmic', 'quadratic','trigonometric'], 'cooling_schedule must be either "linear", "exponential", "logarithmic", "quadratic",or "trigonometric"'
        
        self.coords = coords
        self.N = len(coords)
        self.T = math.sqrt(self.N) if T == -1 else T
        self.T_save = self.T  # save inital T to reset if batch annealing is used
        self.stopping_temp = 1e-8 if stopping_T == -1 else stopping_T
        self.stopping_iter = 100000 if stopping_iter == -1 else stopping_iter
        self.cooling_schedule = cooling_schedule
        self.iteration = 1
        self.nodes = [i for i in range(self.N)]
        self.best_solution = None
        self.best_fitness = float("Inf")
        self.fitness_list = []
        self.bestfit_list = []
        
        
        # initialize cooling schedule
        if self.cooling_schedule == 'linear':
            if alpha != None:
                self.update_t = self.cooling_linear_m
                self.cooling_schedule = 'linear multiplicative cooling'
                self.alpha = alpha
            elif alpha == None:
                self.update_t = self.cooling_linear_a
                self.cooling_schedule = 'linear additive cooling'

        elif self.cooling_schedule == 'quadratic':
            if alpha != None:
                self.update_t = self.cooling_quadratic_m
                self.cooling_schedule = 'quadratic multiplicative cooling'
                self.alpha = alpha
            elif alpha == None:
                self.update_t = self.cooling_quadratic_a
                self.cooling_schedule = 'quadratic additive cooling'

        elif self.cooling_schedule == 'exponential':
            if alpha != None: 
                self.update_t = self.cooling_exponential_m
                self.cooling_schedule = 'exponential multiplicative cooling'
                self.alpha =  alpha              
            elif alpha == None:
                self.update_t = self.cooling_exponential_a
                self.cooling_schedule = 'exponential additive cooling' 
            
        elif self.cooling_schedule == 'trigonometric':
            if alpha != None: 
                print("This is an additive cooling schedule. Alpha value will be igonored!")
                self.update_t = self.cooling_trigonometric_a
                self.cooling_schedule = 'trignometric additive cooling'                
            elif alpha == None:
                self.update_t = self.cooling_trigonometric_a
                self.cooling_schedule = 'trignometric additive cooling'
        
        if self.cooling_schedule == 'logarithmic':
            if alpha == None: self.alpha =  1.1
            else: self.alpha = alpha
            self.update_t = self.cooling_logarithmic_m

    def initial_solution(self, samp_method='replace'):    # Get initial solution using nearest neighbor
        self.s = self.nodes
        cur_node = None
        if samp_method == 'no replace':
            cur_node = random.choice(s)
            self.s.remove(cur_node)
        else:
            cur_node = random.choice(self.nodes)  # start from a random node
        
        solution = [cur_node]

        free_nodes = set(self.nodes)
        free_nodes.remove(cur_node)
        while free_nodes:
            next_node = min(free_nodes, key=lambda x: self.dist(cur_node, x))  # nearest neighbour
            free_nodes.remove(next_node)
            solution.append(next_node)
            cur_node = next_node

        cur_fit = self.fitness(solution)
        if cur_fit < self.best_fitness:  # If best found so far, update best fitness
            self.best_fitness = cur_fit
            self.bestfit_list.append(cur_fit)
            self.best_solution = solution
        self.fitness_list.append(cur_fit)
        return solution, cur_fit

    def dist(self, node_0, node_1):    # Calculate Euclidean distance between two nodes
        coord_0, coord_1 = self.coords[node_0], self.coords[node_1]
        return math.sqrt((coord_0[1] - coord_1[1]) ** 2 + (coord_0[2] - coord_1[2]) ** 2)

    def fitness(self, solution):    # Calculate total distance of current solution path
        cur_fit = 0
        for i in range(self.N):
            cur_fit += self.dist(solution[i % self.N], solution[(i + 1) % self.N])
        return cur_fit

    def p_accept(self, candidate_fitness):    # Acceptance probability/Metropolis criterion ???
        return math.exp(-(candidate_fitness - self.cur_fitness) / self.T)

    def accept(self, candidate):    # Accept with probability 1 if better than current, accept with probability
                                    # p_accept(...) if worse
        candidate_fitness = self.fitness(candidate)
        if candidate_fitness < self.cur_fitness:
            self.cur_fitness, self.cur_solution = candidate_fitness, candidate
            if candidate_fitness < self.best_fitness:
                self.best_fitness, self.best_solution = candidate_fitness, candidate
                self.bestfit_list.append(candidate_fitness)
        else:
            if random.random() < self.p_accept(candidate_fitness):
                self.cur_fitness, self.cur_solution = candidate_fitness, candidate

    def anneal(self, reheat=-1,samp_method='replace',batch=-1):    # Execute SA algorithm
        assert samp_method in ['replace','no replace'], 'sampling method must be either "replace" or "no replace"'
        # Initialize with the greedy solution.
        if reheat==1:
            self.cur_solution = self.best_solution
            self.cur_fitness = self.best_fitness
        elif samp_method=='no replace':   
            self.cur_solution, self.cur_fitness = self.initial_solution(samp_method)
        else:
            self.cur_solution, self.cur_fitness = self.initial_solution()

        print("Starting annealing.")
        while self.T >= self.stopping_temp and self.iteration < self.stopping_iter:
            candidate = list(self.cur_solution)
            l = random.randint(2, self.N - 1)
            i = random.randint(0, self.N - l)
            candidate[i : (i + l)] = reversed(candidate[i : (i + l)])
            self.accept(candidate)
            self.T = self.update_t(self.iteration)
            self.iteration += 1

            self.fitness_list.append(self.cur_fitness)
           
        if batch==1:
            print("Best fitness obtained: ", self.best_fitness)
        else:    
            print("Best fitness obtained: ", self.best_fitness)
            improvement = 100 * (self.fitness_list[0] - self.best_fitness) / (self.fitness_list[0])
            print(f"Improvement over greedy heuristic: {improvement : .2f}%")
        
    def reheat(self):    # Reheat and restart SA with last best solution as starting point
        self.cur_solution, self.cur_fitness = self.initial_solution()
        hist = []
        prev_bestfit=float('inf')
        loop = 1
        while self.best_fitness < prev_bestfit:
            prev_bestfit = self.best_fitness 
            print(f"Iteration {loop}/ -------------------------------")
            self.T = self.T_save
            self.iteration += 1
            self.anneal(reheat=1)
            hist.append(prev_bestfit)
            loop +=1
            
        return hist
            
    def batch_anneal(self, times=10, samp_method='replace'):    # Execute SA algorithm with random initial solutions
        assert samp_method in ['replace','no replace'], 'sampling method must be either "replace" or "no replace"'
        hist = []
        for i in range(1, times + 1):
            print(f"Iteration {i}/{times} -------------------------------")
            self.T = self.T_save
            self.iteration = +1
            self.anneal(samp_method,batch=1)
            hist.append(self.best_fitness)
        
        return hist

    def visualize_routes(self):    # Visualize TSP route
        plotTSP([self.best_solution], self.coords)

    def plot_learning(self):    # Plot fitness through iterations
        fig = plt.figure(figsize=(4,3),dpi=200)
        plt.plot([i for i in range(len(self.fitness_list))], self.fitness_list,linewidth=0.6,label="Tested distance")
        for i,j in enumerate(self.bestfit_list):
            if i==0:
                plt.axhline(y=j,linewidth=0.6,xmin=i/len(self.bestfit_list),xmax=(i+1)/len(self.bestfit_list),color='r',label="Shortest Distance")
            else:
                plt.axhline(y=j,linewidth=0.6,xmin=i/len(self.bestfit_list),xmax=(i+1)/len(self.bestfit_list),color='r')
                
        plt.ylabel("Distance")
        plt.xlabel("Iteration")
        plt.legend(bbox_to_anchor=(1, 1))
        plt.show()

    # linear additive cooling
    def cooling_linear_a(self, step):
        return self.stopping_temp + (self.T_save - self.stopping_temp) * ((self.stopping_iter - step)/self.stopping_iter)

    # quadratic additive cooling
    def cooling_quadratic_a(self, step):
        return self.stopping_temp + (self.T_save - self.stopping_temp) * ((self.stopping_iter - step)/self.stopping_iter)**2
    
    # exponential additive cooling
    def cooling_exponential_a(self, step):
        return self.stopping_temp + (self.T_save - self.stopping_temp)*(1/(1+np.exp((2*np.log(self.T_save - self.stopping_temp)/self.stopping_iter)*(step-(self.stopping_iter/2)))))

    # linear multiplicative cooling
    def cooling_linear_m(self, step):
        return self.T_save /  (1 + self.alpha * step)
    
    # quadratic multiplicative cooling
    def cooling_quadratic_m(alpha,step):
        return T_start/(1 + (alpha * step**2))
    
    # exponential multiplicative cooling
    def cooling_exponential_m(self, step):
        return self.T_save * self.alpha**step
    
    # trignometric additive cooling
    def cooling_trigonometric_a(self, step):
        return self.stopping_temp + 0.5*(self.T_save - self.stopping_temp) * (1+ np.cos((step*np.pi)/self.stopping_iter))
    
    # logarithmical multiplicative cooling
    def cooling_logarithmic_m(self, step):
        return self.T_save / (1+(self.alpha * np.log(step + 1)))
              
        
def plotTSP(paths, points):    # Plot TSP route using list of lists with different orders in which nodes
                                            # were visited (paths), coordinates of nodes (points), and number of
                                            # paths that are in the path list (num_iters)

    # Unpack the primary TSP path and transform it into a list of ordered coordinates
    plt.style.use('default')
    x = []; y = []
    for i in paths[0]:
        x.append(points[i][1])
        y.append(points[i][2])

    fig = plt.figure(figsize=(4,3),dpi=200)
    plt.plot(x, y, 'bo',  ms=1,label='Nodes')

    # Set a scale for the arrow heads 
    a_scale = float(max(x))/float(100)

    # Draw the primary path for the TSP problem
    plt.arrow(x[-1], y[-1], (x[0] - x[-1]), (y[0] - y[-1]), head_width = a_scale,
            color ='r', length_includes_head=True,label='Starting node at arrow head')
    for i in range(0,len(x)-1):
        if i==0:
            plt.arrow(x[i], y[i], (x[i+1] - x[i]), (y[i+1] - y[i]), head_width = a_scale,
                color = 'grey', length_includes_head = True,label='Travel route')
        else:
            plt.arrow(x[i], y[i], (x[i+1] - x[i]), (y[i+1] - y[i]), head_width = a_scale,
                color = 'grey', length_includes_head = True)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(prop={'size': 6},bbox_to_anchor=(1, 1),facecolor='white')
    plt.show()

## Intialise coordinates and define function to calculate some statistics.

In [ ]:
def stats(bestfits, best):
    mean = np.mean(bestfits)
    error = st.sem(bestfits)
    intv95 = st.norm.interval(alpha=0.95, loc=mean, scale=error)
    print(f'\nmean = {mean}, error = {error}, lower bound: {intv95[0]}, upper bound: {intv95[1]}')
    fig = plt.figure(figsize=(4,3),dpi=200)
    plt.hist(bestfits, bins= 'auto', density=True,color='grey',label="Simulation Results with 50 simulations")   
    plt.axvline(x=intv95[0],color='red',linestyle='--',linewidth=0.8,label='Lower bound of 95% Confidence Interval')
    plt.axvline(x=intv95[1],color='green',linestyle='--',linewidth=0.8,label='Upper bound of 95% Confidence Interval')
    plt.axvline(x=best, color='orange',linestyle='dashdot',label='Shortest distance')
    plt.xlabel("Distance")
    plt.ylabel("Frequency of Occurrence")
    plt.legend(prop={'size': 6},bbox_to_anchor=(1, 1))
    plt.show()
    return mean, error

coords = []
with open('a280.tsp.txt', "r") as f:

    for i,line in enumerate(f.readlines()):
        if i<=5 or i>285 : pass
        else:
            line = [float(x.replace("\n", " ")) for x in line.split()]
            coords.append(line)

## Sampling with or without replacement

In [ ]:
start_time = time.time()
sa_norep = SimAnneal(coords,cooling_schedule='exponential',T=20,stopping_T=1e-6,alpha=0.999,stopping_iter=100000)
bestfits = sa_norep.batch_anneal(samp_method='replace')  
simtime = time.time()-start_time 
best =  sa_norep.best_fitness
sa_norep.visualize_routes()
sa_norep.plot_learning()
mean, error = stats(bestfits,best)

## Reheating
- Since reheat continues only when subsequent solutions are better than the previous ones, we run a loop. 

In [ ]:
start_time = time.time()
bestfits = []
for i in range(5):
    print('\033[1m'+"...........Reheat attempt {}........... ".format(i+1)+'\033[0m')
    sa_rh = SimAnneal(coords)
    bestfits.append(sa_rh.reheat())  
    
simtime = time.time()-start_time
print("\nBestfits:", bestfits,"\nSimtime: " , simtime," seconds")

## Test for determining starting temp(T) and maximum iterations <br> Parameters used T = {50,20}, stopping_iter = {100000,50000} , Total simulations = 30. <br><br>The tests are carried for different cooling schedules (the summary statistics are tabulated in a pretty table in every 4th cell):

### Linear Additive cooling schedule

In [ ]:
x = PrettyTable()
x.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]
start_time = time.time()
sa_lin1 = SimAnneal(coords,cooling_schedule='linear',T=100)
bestfits = sa_lin1.batch_anneal(times=30)  
simtime = time.time()-start_time 
best =  sa_lin1.best_fitness
sa_lin1.visualize_routes()
sa_lin1.plot_learning()
mean, error = stats(bestfits,best)
x.add_row(["T=100,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_lin2 = SimAnneal(coords,cooling_schedule='linear',T=20)
bestfits = sa_lin2.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lin2.best_fitness
sa_lin2.visualize_routes()
sa_lin2.plot_learning()
mean, error = stats(bestfits,best)
x.add_row(["T=20,Maximum Iterations=100,000",best,mean, error, simtime])  

In [ ]:
start_time = time.time()

sa_lin3 = SimAnneal(coords,cooling_schedule='linear',T=50,stopping_iter=50000)
bestfits = sa_lin3.batch_anneal(times=30)  
simtime = time.time()-start_time 
best =  sa_lin3.best_fitness
sa_lin3.visualize_routes()
sa_lin3.plot_learning()
mean, error = stats(bestfits,best)
x.add_row(["T=50,Maximum Iterations=50,000",best,mean, error, simtime]) 

In [ ]:
start_time = time.time()

sa_lin4 = SimAnneal(coords,cooling_schedule='linear',T=20,stopping_iter=50000)
bestfits = sa_lin4.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lin4.best_fitness
sa_lin4.visualize_routes()
sa_lin4.plot_learning()
mean, error = stats(bestfits,best)
x.add_row(["T=20,Maximum Iterations=50,000",best,mean, error, simtime]) 
print(x)

### Tests for Exponential Additive

In [ ]:
exp_a = PrettyTable()
exp_a.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_ea1 = SimAnneal(coords,cooling_schedule='exponential',T=50)
bestfits = sa_ea1.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_ea1.best_fitness
sa_ea1.visualize_routes()
sa_ea1.plot_learning()
mean, error = stats(bestfits,best)
exp_a.add_row(["T=50,Maximum Iterations=100,000",best,mean, error, simtime]) 

In [ ]:
start_time = time.time()

sa_ea2 = SimAnneal(coords,cooling_schedule='exponential',T=20)
bestfits = sa_ea2.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_ea2.best_fitness
sa_ea2.visualize_routes()
sa_ea2.plot_learning()
mean, error = stats(bestfits,best)
exp_a.add_row(["T=20,Maximum Iterations=100,000",best,mean, error, simtime]) 

In [ ]:
start_time = time.time()

sa_ea3 = SimAnneal(coords,cooling_schedule='exponential',T=50,stopping_iter=50000)
bestfits = sa_ea3.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_ea3.best_fitness
sa_ea3.visualize_routes()
sa_ea3.plot_learning()
mean, error = stats(bestfits,best)
exp_a.add_row(["T=50,Maximum Iterations=50,000",best,mean, error, simtime]) 

In [ ]:
start_time = time.time()

sa_ea4 = SimAnneal(coords,cooling_schedule='exponential',T=20,stopping_iter=50000)
bestfits = sa_ea4.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_ea4.best_fitness
sa_ea4.visualize_routes()
sa_ea4.plot_learning()
mean, error = stats(bestfits,best)
exp_a.add_row(["T=20,Maximum Iterations=50,000",best,mean, error, simtime]) 
print(exp_a)

## Linear Multiplicative Tests

In [ ]:
lintab = PrettyTable()
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab1 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=50)
bestfits = sa_lintab1.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab1.best_fitness
sa_lintab1.visualize_routes()
sa_lintab1.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=1,T=50,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab2 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=20)
bestfits = sa_lintab2.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab2.best_fitness
sa_lintab2.visualize_routes()
sa_lintab2.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=1,T=20,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab3 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=50,stopping_iter=50000)
bestfits = sa_lintab3.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab3.best_fitness
sa_lintab3.visualize_routes()
sa_lintab3.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=1,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab4 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_lintab4.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab4.best_fitness
sa_lintab4.visualize_routes()
sa_lintab4.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=1,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

### alpha = 2

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab5 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=50)
bestfits = sa_lintab5.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab5.best_fitness
sa_lintab5.visualize_routes()
sa_lintab5.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=2,T=50,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab6 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=20)
bestfits = sa_lintab6.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab6.best_fitness
sa_lintab6.visualize_routes()
sa_lintab6.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=2,T=20,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab7 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=50,stopping_iter=50000)
bestfits = sa_lintab7.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab7.best_fitness
sa_lintab7.visualize_routes()
sa_lintab7.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=2,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
lintab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab8 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_lintab8.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_lintab8.best_fitness
sa_lintab8.visualize_routes()
sa_lintab8.plot_learning()
mean, error = stats(bestfits,best)
lintab.add_row(["alpha=2,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
print(lintab)

## Exponential Multiplicative Tests

In [ ]:
exptab = PrettyTable()
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab1 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=50)
bestfits = sa_exptab1.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab1.best_fitness
sa_exptab1.visualize_routes()
sa_exptab1.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=1,T=50,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab2 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=20)
bestfits = sa_exptab2.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab2.best_fitness
sa_exptab2.visualize_routes()
sa_exptab2.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=1,T=20,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab3 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=50,stopping_iter=50000)
bestfits = sa_exptab3.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab3.best_fitness
sa_exptab3.visualize_routes()
sa_exptab3.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=1,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab4 = SimAnneal(coords,alpha=1,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_exptab4.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab4.best_fitness
sa_exptab4.visualize_routes()
sa_exptab4.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=1,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

### alpha = 2

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab5 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=50)
bestfits = sa_exptab5.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab5.best_fitness
sa_exptab5.visualize_routes()
sa_exptab5.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=2,T=50,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab6 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=20)
bestfits = sa_exptab6.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab6.best_fitness
sa_exptab6.visualize_routes()
sa_exptab6.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=2,T=20,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab7 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=50,stopping_iter=50000)
bestfits = sa_exptab7.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab7.best_fitness
sa_exptab7.visualize_routes()
sa_exptab7.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=2,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
exptab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab8 = SimAnneal(coords,alpha=2,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_exptab8.batch_anneal(times=30)  
simtime = time.time()-start_time
best =  sa_exptab8.best_fitness
sa_exptab8.visualize_routes()
sa_exptab8.plot_learning()
mean, error = stats(bestfits,best)
exptab.add_row(["alpha=2,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
print(exptab)

## Logarithmic Multiplicative Tests

### alpha = 0.85

In [ ]:
logtab = PrettyTable()
logtab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_logtab1 = SimAnneal(coords,alpha=0.85,cooling_schedule='logarithmic',T=50)
bestfits = sa_logtab1.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab1.best_fitness
sa_logtab1.visualize_routes()
sa_logtab1.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=0.85,T=50,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab2 = SimAnneal(coords,alpha=0.85,cooling_schedule='logarithmic',T=20)
bestfits = sa_logtab2.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab2.best_fitness
sa_logtab2.visualize_routes()
sa_logtab2.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=0.85,T=20,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab3 = SimAnneal(coords,alpha=0.85,cooling_schedule='logarithmic',T=50,stopping_iter=50000)
bestfits = sa_logtab3.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab3.best_fitness
sa_logtab3.visualize_routes()
sa_logtab3.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=0.85,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab4 = SimAnneal(coords,alpha=0.85,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_logtab4.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab4.best_fitness
sa_logtab4.visualize_routes()
sa_logtab4.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=0.85,T=20,Maximum Iterations=50,000",best,mean, error, simtime])

### alpha = 1.3

In [ ]:
logtab.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_logtab5 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=50)
bestfits = sa_logtab5.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab5.best_fitness
sa_logtab5.visualize_routes()
sa_logtab5.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=1.3,T=50,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab6 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=20)
bestfits = sa_logtab6.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab6.best_fitness
sa_logtab6.visualize_routes()
sa_logtab6.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=1.3,T=20,Maximum Iterations=100,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab7 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=50,stopping_iter=50000)
bestfits = sa_logtab7.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab7.best_fitness
sa_logtab7.visualize_routes()
sa_logtab7.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=1.3,T=50,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab8 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_logtab8.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab8.best_fitness
sa_logtab8.visualize_routes()
sa_logtab8.plot_learning()
mean, error = stats(bestfits,best)
logtab.add_row(["alpha=1.3,T=20,Maximum Iterations=50,000",best,mean, error, simtime])

In [ ]:
print(logtab)

## Test for determining stopping temp(stopping_T) :

## Linear Additive

In [ ]:
lintab_temp = PrettyTable()
lintab_temp.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintab_temp1 = SimAnneal(coords,cooling_schedule='linear',T=20,stopping_iter=50000)
bestfits = sa_lintab_temp1.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_lintab_temp1.best_fitness
sa_lintab_temp1.visualize_routes()
sa_lintab_temp1.plot_learning()
mean, error = stats(bestfits,best)
lintab_temp.add_row(["alpha=,T=,Maximum Iterations=,Stopping Temperature=10^-8",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_lintab_temp2 = SimAnneal(coords,cooling_schedule='linear',T=20,stopping_iter=50000,stopping_T=1e-6)
bestfits = sa_lintab_temp2.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_lintab_temp2.best_fitness
sa_lintab_temp2.visualize_routes()
sa_lintab_temp2.plot_learning()
mean, error = stats(bestfits,best)
lintab_temp.add_row(["alpha=,T=,Maximum Iterations=,Stopping Temperature=10^-6",best,mean, error, simtime])

In [ ]:
print(lintab_temp)

## Linear Exponential

In [ ]:
exptab_temp = PrettyTable()
exptab_temp.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptab_temp1 = SimAnneal(coords,cooling_schedule='exponential',T=20,stopping_iter=50000)
bestfits = sa_exptab_temp1.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_exptab_temp1.best_fitness
sa_exptab_temp1.visualize_routes()
sa_exptab_temp1.plot_learning()
mean, error = stats(bestfits,best)
exptab_temp.add_row(["alpha=,T=,Maximum Iterations=,Stopping Temperature=10^-8",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_exptab_temp2 = SimAnneal(coords,cooling_schedule='exponential',T=20,stopping_iter=50000,stopping_T=1e-6)
bestfits = sa_exptab_temp2.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_exptab_temp2.best_fitness
sa_exptab_temp2.visualize_routes()
sa_exptab_temp2.plot_learning()
mean, error = stats(bestfits,best)
exptab_temp.add_row(["alpha=,T=,Maximum Iterations=,Stopping Temperature=10^-6",best,mean, error, simtime])

In [ ]:
print(exptab_temp)

## Linear Multiplicative

In [ ]:
lintabmult_temp = PrettyTable()
lintabmult_temp.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_lintabmult_temp1 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=20)
bestfits = sa_lintabmult_temp1.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_lintabmult_temp1.best_fitness
sa_lintabmult_temp1.visualize_routes()
sa_lintabmult_temp1.plot_learning()
mean, error = stats(bestfits,best)
lintabmult_temp.add_row(["alpha=1.3,T=20,Maximum Iterations=100000,Stopping Temperature=10^-8",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_lintabmult_temp2 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=20,stopping_T=1e-6)
bestfits = sa_lintabmult_temp2.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_lintabmult_temp2.best_fitness
sa_lintabmult_temp2.visualize_routes()
sa_lintabmult_temp2.plot_learning()
mean, error = stats(bestfits,best)
lintabmult_temp.add_row(["alpha=1.3,T=50,Maximum Iterations=100000, Stopping Temperature=10^-6",best,mean, error, simtime])

In [ ]:
print(lintabmult_temp)

## Exponential Multiplicative

In [ ]:
exptabmult_temp = PrettyTable()
exptabmult_temp.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_exptabmult_temp1 = SimAnneal(coords,alpha=0.999,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_exptabmult_temp1.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_exptabmult_temp1.best_fitness
sa_exptabmult_temp1.visualize_routes()
sa_exptabmult_temp1.plot_learning()
mean, error = stats(bestfits,best)
exptabmult_temp.add_row(["alpha=0.999,T=20,Maximum Iterations=50000,Stopping Temperature=10^-8",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_exptabmult_temp2 = SimAnneal(coords,alpha=0.999,cooling_schedule='logarithmic',T=20,stopping_T=1e-6,stopping_iter=50000)
bestfits = sa_exptabmult_temp2.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_exptabmult_temp2.best_fitness
sa_exptabmult_temp2.visualize_routes()
sa_exptabmult_temp2.plot_learning()
mean, error = stats(bestfits,best)
exptabmult_temp.add_row(["alpha=0.999,T=20,Maximum Iterations=50000, Stopping Temperature=10^-6",best,mean, error, simtime])

In [ ]:
print(exptabmult_temp)

## Logarithmic

In [ ]:
logtab_temp = PrettyTable()
logtab_temp.field_names = ["Parameters", "Best Solution", "Mean", "Error", "Simulation Time(in secs)"]

start_time = time.time()

sa_logtab_temp1 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=20,stopping_iter=50000)
bestfits = sa_logtab_temp1.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab_temp1.best_fitness
sa_logtab_temp1.visualize_routes()
sa_logtab_temp1.plot_learning()
mean, error = stats(bestfits,best)
logtab_temp.add_row(["alpha=1,3,T=20,Maximum Iterations=50,000,Stopping Temperature=10^-8",best,mean, error, simtime])

In [ ]:
start_time = time.time()

sa_logtab_temp2 = SimAnneal(coords,alpha=1.3,cooling_schedule='logarithmic',T=20,stopping_T=1e-6,stopping_iter=50000)
bestfits = sa_logtab_temp2.batch_anneal(times=50)  
simtime = time.time()-start_time
best =  sa_logtab_temp2.best_fitness
sa_logtab_temp2.visualize_routes()
sa_logtab_temp2.plot_learning()
mean, error = stats(bestfits,best)
logtab_temp.add_row(["alpha=1.3,T=20,Maximum Iterations=50,000, Stopping Temperature=10^-6",best,mean, error, simtime])

In [ ]:
print(logtab_temp)